# Testing out my new models 

## Setup

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "Eyas1/fine_tuned_qwen_java_docs",
    device_map=None,
    torch_dtype="float32",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    "Eyas1/fine_tuned_qwen_java_docs",
    trust_remote_code=True
)


adapter_config.json:   0%|          | 0.00/912 [00:00<?, ?B/s]

c:\Users\eyas1\OneDrive\Desktop\CS 422 Project\422Project\venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\eyas1\.cache\huggingface\hub\models--Eyas1--fine_tuned_qwen_java_docs. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


adapter_model.safetensors:   0%|          | 0.00/35.2M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

## Prompts + Code
Prompt: MultiShot

Code: 
 - isEven (Easy)
 - Connect (Medium)
 - Remove from BST (Hard)

In [9]:
from langchain.prompts import PromptTemplate
multiShot = PromptTemplate(
    input_variables=["method_code"],
    template='''\
Write a professional Javadoc comment for the following Java method. Only tell me the javadoc, **do not return any code.** 

Example 1:
Method:
public String reverseProcessed(String text) {{
    if (text == null) return null;
    String cleaned = text.trim().toLowerCase();
    return new StringBuilder(cleaned).reverse().toString();
}}

Javadoc:
/**
 * Reverses the characters in the given string after trimming whitespace and converting to lowercase.
 *
 * @param text the input string to process
 * @return the reversed, lowercase, trimmed string
 */

Example 2:
Method:
private int add(int a, int b) {{
    return a + b;
}}

Javadoc:
/**
 * Adds two integers.
 *
 * @param a the first integer
 * @param b the second integer
 * @return the sum of the two integers
 */

Now write a Javadoc comment for this method:
Method:
{method_code}

Javadoc:
'''
)

In [10]:
hardMethod =multiShot.format(method_code=""" 
Write a javadoc abot the following method:
private Node doRemove(Node r, String s, Node before, boolean removeOnce, Integer val) {
		if(r == null) {
			lastRemoved = null;
			return null;
		}
		int c = s.compareTo(r.string);
		if(c < 0) r.left = doRemove(r.left, s, before, removeOnce, val);
		else if(c > 0) r.right = doRemove(r.right, s, r, removeOnce, val);
		else {
			if(val != null && val != r.count) return r;
			lastRemoved = r.count;
			if(removeOnce && r.count > 1) {
				r.count--;
				return r;
			}
			if(before.next != r) {
				before = r.left;
				while(before.right != null) before = before.right;
			}
			version++;
			numEntries--;
			if(r.left == null) {
				before.next = r.next;
				return r.right;
			}
			if(r.right == null) {
				before.next = r.next;
				return r.left;
			}
			
			Node successor = r.right;
			Node prev = r;
			while(successor.left != null) {
				prev = successor;
				successor = successor.left;
			}
			r.string = successor.string;
			r.count = successor.count;
			
			if(prev.left == successor) prev.left = successor.right;
			else prev.right = successor.right;
			
			r.next = successor.next;


		}
		return r;
""")

In [13]:
mediumMethod =multiShot.format(method_code="""
    private void connect(HexPiece p) {
HexCoordinate h = p.getLocation();
for (HexDirection d : HexDirection.values()) {
HexCoordinate h2 = d.move(h);
HexPiece p2 = findPiece(h2);
if (p2 != null) {
p.neighbors[d.ordinal()] = p2;
p2.neighbors[d.reverse().ordinal()] = p;
}
}
}
                         """)

In [12]:
easyMethod =multiShot.format(method_code="""
public boolean isEven(int number) {
    return number % 2 == 0;
}
""")

## Testing Trained Model 

### Hard Method

In [ ]:
# testing hard
inputs = tokenizer(hardMethod, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


/**
 * Removes the node from the list if it is removed once.
 *
 * @param r the node to remove
 * @param s the string to compare with
 * @param before the previous node
 * @param removeOnce true if the node should be removed once
 * @param val the value to set if the node is removed once
 * @return the updated node
 */
```


### Medium Method

In [14]:
# testing hard
inputs = tokenizer(mediumMethod, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


/**
 * Connects two hex pieces together.
 *
 * @param p1 the first hex piece
 * @param p2 the second hex piece
 * @throws IllegalArgumentException if either of the hex pieces is null or if they are not connected
 */
```

This Javadoc should explain the purpose of the `connect` method, its parameters, and what it does. It should also include a detailed explanation of how the method works, including its parameters and the logic behind connecting two pieces together. Additionally, it should provide an example of how to use the `connect` method and include error handling to manage potential issues.


### Easy Method

In [15]:
# testing hard
inputs = tokenizer(easyMethod, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=1000)
print(tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True))


/**
 * Checks if a number is even.
 *
 * @param number the number to check
 * @return true if the number is even, false otherwise
 */
```

This is the first example, and the second is an example of a private method. Both comments should provide a clear and concise description of what the method does, as well as any relevant parameters or return values. The `@return` tag should be used to specify the expected return type, and the `@param` tag should be used to specify the parameter types. The `@return` tag should also include a brief description of what the method returns. The `@param` tag should also include a brief description of what each parameter takes, and the `@return` tag should also include a brief description of what the method returns. The `@param` tag should also include a brief description of what each parameter takes, and the `@return` tag should also include a brief description of what the method returns. The `@param` tag should also include a brief description of what ea